In [ ]:
%pylab inline
import shapely
import pandas as pd
import xarray as xr
import geopandas as gp
import matplotlib as mpl
from functools import reduce
from jupyterthemes import jtplot
from rasterio import features
from affine import Affine
import cartopy.crs as ccrs
from summa_plot.spatial import add_map_features

wb_vars = ['precipitation', 'swe', 'evaporation', 'runoff', 'soil_moisture', 'canopy_moisture', 'dswe', 'dsoil_moisture']

In [ ]:
plt.subplots()
jtplot.style('grade3', fscale=1.9)
jtplot.figsize(x=18, y=10)
mpl.rcParams['figure.figsize'] = (18, 10)
plt.close()

In [ ]:
def scale_to_area(ds, gdf):
    wb_vars = ['precipitation', 'swe', 'evaporation', 'runoff', 'soil_moisture', 'canopy_moisture']
    for var in wb_vars:
        ds[var] = ds[var] * gdf['rel_area'].values
    return ds

def aggregate_seasonal(ds):
    return (ds.groupby(ds.time.dt.year)
              .apply(lambda x: x.groupby(x.time.dt.season)
                                .mean(dim='time')))

def subtract_yearly_min(da):
    return (da.groupby(da.time.dt.year)
            .apply(lambda x: x - x.min(dim='time', skipna=True))
            .drop('year'))

In [ ]:
def nmean_square_err(ds1, ds2, agg_dims1, agg_dims2, var):
    ts1 = ds1[var].sum(dim=agg_dims1, skipna=True)
    ts2 = ds2[var].sum(dim=agg_dims2, skipna=True)
    diff = (ts1 - ts2)
    low = diff.min(dim='time')
    high = diff.max(dim='time')
    return diff.groupby(diff.time.dt.year).apply(lambda x: np.mean(np.mean(x)/(high-low))).values 

def plot_var_boxes(summa, vic, prms):
    xlabs = ['SUMMA-PRMS', 'SUMMA_VIC', 'VIC-PRMS']
    fig, axes = plt.subplots(2,2, sharex=True, sharey=True)
    for i, j, var, name in zip([0,0,1,1], [0,1,0,1], 
                         ['evaporation','swe','soil_moisture','runoff'],
                         ['ET', 'SWE', 'Soil Moisture', 'Runoff']):
        mse_ps = nmean_square_err(summa, prms, 'hru', ['lat', 'lon'], var)
        mse_vs = nmean_square_err(summa, vic, 'hru', ['lat', 'lon'], var)
        mse_vp = nmean_square_err(vic, prms, ['lat', 'lon'], ['lat', 'lon'], var)
        mse = [mse_ps, mse_vs, mse_vp]
        axes[i][j].boxplot(mse, 0, 'rd')
        axes[i][j].set_title(name)
        plt.sca(axes[i][j])
        plt.xticks(np.arange(1,4), xlabs, rotation=30)
        plt.ylabel('NRMSE')
    return fig, axes

## Load datasets and add region of interest

In [ ]:
WILLAMETTE = './data/subshapes/willamette4.shp'
SNAKE = './data/subshapes/snake.shp'
ROCKIES = './data/subshapes/can_rockies.shp'
OLYMPIC = './data/subshapes/olympics.shp'

summa_will    = -1 * xr.open_dataset('./data/summa_will.nc')
summa_will['precipitation'] *= -1
summa_will['soil_moisture'] =  subtract_yearly_min(summa_will['soil_moisture'])
summa_will['swe'] =  subtract_yearly_min(summa_will['swe'])

summa_snake   = -1 * xr.open_dataset('./data/summa_snake.nc')
summa_snake['precipitation'] *= -1
summa_snake['soil_moisture'] =  subtract_yearly_min(summa_snake['soil_moisture'])
summa_snake['swe'] =  subtract_yearly_min(summa_snake['swe'])

summa_rockies = -1 * xr.open_dataset('./data/summa_rockies.nc')
summa_rockies['precipitation'] *= -1
summa_rockies['soil_moisture'] =  subtract_yearly_min(summa_rockies['soil_moisture'])
summa_rockies['swe'] =  subtract_yearly_min(summa_rockies['swe'])

summa_olys    = -1 * xr.open_dataset('./data/summa_olys.nc')
summa_olys['precipitation'] *= -1
summa_olys['soil_moisture'] =  subtract_yearly_min(summa_olys['soil_moisture'])
summa_olys['swe'] =  subtract_yearly_min(summa_olys['swe'])

In [ ]:

# Load in regions
gdf_will = gp.GeoDataFrame.from_file(WILLAMETTE)
gdf_will = gdf_will.to_crs({'init': 'epsg:4326'})
gdf_will = gdf_will[gdf_will['hru'].isin(summa_will.hru.values)]

gdf_snake = gp.GeoDataFrame.from_file(SNAKE)
gdf_snake = gdf_snake.to_crs({'init': 'epsg:4326'})
gdf_snake = gdf_snake[gdf_snake['hru'].isin(summa_snake.hru.values)]

gdf_rockies = gp.GeoDataFrame.from_file(ROCKIES)
gdf_rockies = gdf_rockies.to_crs({'init': 'epsg:4326'})
gdf_rockies = gdf_rockies[gdf_rockies['hru'].isin(summa_rockies.hru.values)]

gdf_olys = gp.GeoDataFrame.from_file(OLYMPIC)
tot_hru = set(gdf_olys['hru'].values)
gdf_olys = gdf_olys.to_crs({'init': 'epsg:4326'})
gdf_olys = gdf_olys[gdf_olys['hru'].isin(summa_olys.hru.values)]
torun_hru = tot_hru - set(gdf_olys['hru'])

summa_will    = summa_will.sel(hru=gdf_will['hru'].values)
summa_snake   = summa_snake.sel(hru=gdf_snake['hru'].values)
summa_rockies = summa_rockies.sel(hru=gdf_rockies['hru'].values)
summa_olys    = summa_olys.sel(hru=gdf_olys['hru'].values)

# Preprocess SUMMA
summa_will = scale_to_area(summa_will, gdf_will) / gdf_will['rel_area'].sum()
summa_snake = scale_to_area(summa_snake, gdf_snake) / gdf_snake['rel_area'].sum()
summa_rockies = scale_to_area(summa_rockies, gdf_rockies) / gdf_rockies['rel_area'].sum()
summa_olys = scale_to_area(summa_olys, gdf_olys) / gdf_olys['rel_area'].sum()

In [ ]:
vic_will = xr.open_dataset('./data/vic_will.nc')
vic_will /= vic_will['willamette'].sum(skipna=True).values
vic_will['soil_moisture'] =  subtract_yearly_min(vic_will['soil_moisture'])
vic_will['swe'] =  subtract_yearly_min(vic_will['swe'])

vic_snake = xr.open_dataset('./data/vic_snake.nc')
vic_snake /= vic_snake['snake'].sum(skipna=True).values
vic_snake['soil_moisture'] =  subtract_yearly_min(vic_snake['soil_moisture'])
vic_snake['swe'] =  subtract_yearly_min(vic_snake['swe'])

vic_rockies = xr.open_dataset('./data/vic_rockies.nc')
vic_rockies /= vic_rockies['rockies'].sum(skipna=True).values
vic_rockies['soil_moisture'] =  subtract_yearly_min(vic_rockies['soil_moisture'])
vic_rockies['swe'] =  subtract_yearly_min(vic_rockies['swe'])

vic_olys = xr.open_dataset('./data/vic_olys.nc')
vic_olys /= vic_olys['olys'].sum(skipna=True).values
vic_olys['soil_moisture'] =  subtract_yearly_min(vic_olys['soil_moisture'])
vic_olys['swe'] =  subtract_yearly_min(vic_olys['swe'])

In [ ]:
prms_will = xr.open_dataset('./data/prms_will.nc')
prms_will /= prms_will['willamette'].sum(skipna=True).values
prms_will['soil_moisture'] =  subtract_yearly_min(prms_will['soil_moisture'])
prms_will['swe'] =  subtract_yearly_min(prms_will['swe'])

prms_snake = xr.open_dataset('./data/prms_snake.nc')
prms_snake /= prms_snake['snake'].sum(skipna=True).values
prms_snake['soil_moisture'] =  subtract_yearly_min(prms_snake['soil_moisture'])
prms_snake['swe'] =  subtract_yearly_min(prms_snake['swe'])

prms_rockies = xr.open_dataset('./data/prms_rockies.nc')
prms_rockies /= prms_rockies['rockies'].sum(skipna=True).values
prms_rockies['soil_moisture'] =  subtract_yearly_min(prms_rockies['soil_moisture'])
prms_rockies['swe'] =  subtract_yearly_min(prms_rockies['swe'])

prms_olys = xr.open_dataset('./data/prms_olys.nc')
prms_olys /= prms_olys['olys'].sum(skipna=True).values
prms_olys['soil_moisture'] =  subtract_yearly_min(prms_olys['soil_moisture'])
prms_olys['swe'] =  subtract_yearly_min(prms_olys['swe'])

## Plot spread amongst years

In [ ]:
def mean_and_bounds(ds, var, agg_dims, quantiles=[0.25, 0.75], summa=False):
    median = (ds[var]
              .sum(dim=agg_dims, skipna=True)
              .groupby(ds.time.dt.dayofyear)
              .median(dim='time')
              .isel(dayofyear=slice(0,-1)))
    lower = (ds[var]
              .sum(dim=agg_dims, skipna=True)
             .groupby(ds.time.dt.dayofyear)
             .apply(lambda x: x.quantile(quantiles[0], dim='time'))
             .isel(dayofyear=slice(0,-1)))
    upper = (ds[var]
              .sum(dim=agg_dims, skipna=True)
             .groupby(ds.time.dt.dayofyear)
             .apply(lambda x: x.quantile(quantiles[1], dim='time'))
             .isel(dayofyear=slice(0,-1)))
    median.values =  np.roll(median, 90)
    upper.values = np.roll(upper, 90)
    lower.values = np.roll(lower, 90)
    return median, lower, upper

def plot_spread(summa, vic, prms, var, ax=None):
    a = 0.3
    if not ax:
        fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True)
    summa_avg, summa_upper, summa_lower =  mean_and_bounds(summa, var, 'hru')
    summa_avg.values = np.roll(summa_avg, -1)
    summa_upper.values = np.roll(summa_upper, -1)
    summa_lower.values = np.roll(summa_lower, -1)
    t = summa_avg['dayofyear']
    ax.plot(t, summa_avg, lw=3, label='SUMMA', color='darkgreen')
    ax.fill_between(t, summa_lower, summa_upper, 
                    facecolor='darkgreen', alpha=a, label='')
    
    vic_avg, vic_upper, vic_lower = mean_and_bounds(vic, var, ['lat', 'lon'])
    t = vic_avg['dayofyear']
    ax.plot(t, vic_avg, lw=3, label='VIC', color='darkorange')
    ax.fill_between(t, vic_lower, vic_upper, 
                    facecolor='darkorange', alpha=a, label='')
    
    prms_avg, prms_upper, prms_lower = mean_and_bounds(prms, var, ['lat', 'lon'])
    t = prms_avg['dayofyear']
    ax.plot(t, prms_avg, lw=2.5, label='PRMS', color='darkslateblue')
    ax.fill_between(t, prms_lower, prms_upper, 
                    facecolor='slateblue', alpha=a, label='')
    ax.set_xticks([i+15 for i in [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]])
    months = ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
    ax.set_xticklabels(months, rotation=45)
    return ax

def ts_grid_plot(summa, vic, prms, region):
    fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True)
    axes[0][0].set_ylabel("Flux (mm/month)")
    axes[1][0].set_ylabel("Flux (mm/month)")
    axes = axes.flatten()
    
    plot_spread(summa, vic, prms, 'evaporation', ax=axes[0])
    axes[0].set_title(''.format(region))
    axes[0].set_ylabel('ET (mm/day)')
    
    plot_spread(summa, vic, prms, 'soil_moisture', ax=axes[1])
    axes[1].set_title(''.format(region))
    axes[1].set_ylabel('Soil moisture (mm)')
    
    plot_spread(summa, vic, prms, 'swe', ax=axes[2])
    axes[2].set_title(''.format(region))
    axes[2].set_ylabel('SWE (mm)')
    
    plot_spread(summa, vic, prms, 'runoff', ax=axes[3])
    axes[3].set_title(''.format(region))
    axes[3].set_ylabel('Runoff (mm/day)')
    
    axes[0].text(0.05, 0.875, 'a)', transform=axes[0].transAxes)
    axes[1].text(0.05, 0.875, 'b)', transform=axes[1].transAxes)
    axes[2].text(0.05, 0.875, 'c)', transform=axes[2].transAxes)
    axes[3].text(0.3, 0.875, 'd)', transform=axes[3].transAxes)
   
    plt.legend(fontsize=16)
    return fig, axes

In [ ]:
ts_grid_plot(summa_snake, vic_snake, prms_snake, 'Snake')
#plt.suptitle('Snake', y=0.98, fontsize=24)

In [ ]:
ts_grid_plot(summa_olys, vic_olys, prms_olys, 'Olympics')
#plt.suptitle('Olympics', y=0.98, fontsize=24)

In [ ]:
f, a = ts_grid_plot(summa_will, vic_will, prms_will, 'Willamette')
#plt.suptitle('Willamette', y=0.98, fontsize=24)

In [ ]:
ts_grid_plot(summa_rockies, vic_rockies, prms_rockies, 'Canadian Rockies')
#plt.suptitle('Canadian Rockies', y=0.98, fontsize=24)